In [2]:
def run_inference(experiment_name_dir, model_arch):
    LOGGER.info('========== Running inference ==========')
    base_experiment_filename = Config.save_dir + f'/{experiment_name_dir}/{model_arch}_fold'    

    model_states = [torch.load(base_experiment_filename + f'{fold}.pth')['model']
                    for fold in range(Config.fold_num)]
    
    # temporary try out
    for i in range(4):
        model_states.append(torch.load(Config.save_dir + f'/exp1_88_Adam/{Config.model_arch}_fold{i}.pth')['model'])

    if not kaggle: 
        loader = holdout_dataloader
        num_samples = len(holdout_df)
    else: 
        loader = test_dataloader 
        num_samples = len(test_df)

    inference_start = time.time()

    predictions = ensemble_inference(model_states, Config.model_arch, 
                            data_df.label.nunique(), loader, num_samples, device, mode='vote')

    inference_elapsed = time.time() - inference_start
    LOGGER.info(f"Inference time: {str(timedelta(seconds=inference_elapsed))}")

    if not kaggle:
        holdout_accuracy = accuracy_score(y_true=holdout_targets, y_pred=predictions)
        LOGGER.info(f"Ensemble model holdout accuracy: {holdout_accuracy}")

    # submission
    if kaggle:
        submission = pd.DataFrame()
        submission['image_id'] = test_df['image_id']
        submission['label'] = predictions
        submission.to_csv('submission.csv', index=False)
    
    return resultsStore